In [1]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive/')

# ディレクトリの移動
%cd /content/drive/My Drive/colony

# ディレクトリ内のデータの確認
%ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/colony
colony_class_index.json  colony_mobilenet_v3_large.ipynb  model/
colony_data/             colony_test.ipynb


In [20]:
# ライブラリのインポート
import torchvision.models as models
from torchvision import transforms
import torch
from torch import nn

import numpy as np

from PIL import Image

import json

'''
初期設定
'''

# 入力画像
image_file_path = './colony_data/test/211215-0027.jpg'

# クラス数の定義
num_classes = 2

# リサイズする画像サイズ
photo_size = 300

# 保存先のパス
save_path = '/content/drive/My Drive/colony/model/47_logloss_0.0195.pth'

In [21]:
# transformの設定
transform = {
    'val': transforms.Compose([
        transforms.Resize(photo_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
}

In [22]:
# モデルの定義
model = models.mobilenet_v3_large()
fc_in_features = model.classifier[3].in_features # 最終レイヤー関数の次元数
model.fc = nn.Linear(fc_in_features, num_classes) # 最終レイヤー関数の付け替え
model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False

In [23]:
class ColonyPredictor():
  """
  モデルの出力からラベルを求める。

  Attributes
  ----------
  class_index : dictionary
      クラスindexとラベル名を対応させた辞書型変数。
  """
  def __init__(self, class_index):
        self.class_index = class_index

  def predict_max(self, out):
      """
      確率最大のラベル名を取得する。

      Parameters
      ----------
      out : torch.Size([1, 2])
          modelからの出力。

      Returns
      -------
      predicted_label_name : str
          最も予測確率が高いラベルの名前
      """
      maxid = np.argmax(out.detach().numpy())
      predicted_label_name = self.class_index[str(maxid)][1]

      return predicted_label_name

In [24]:
# コロニーのラベル情報をロードし、辞書型変数を生成
colony_class_index = json.load(open('./colony_class_index.json', 'r'))

#  ColonyPredictorのインスタンス
predictor = ColonyPredictor(colony_class_index)

# 入力画像の読み込み
img = Image.open(image_file_path)

# 画像の前処理
img_transformed = transform['val'](img)
input = img_transformed.unsqueeze_(0)

# モデルに入力し、ラベルを出力
output = model(input)
result = predictor.predict_max(output)
print("入力画像の予測結果：", result)

入力画像の予測結果： non-colony
